In [1]:
from pathlib import Path
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import torch
import SimpleITK as sitk
import numpy as np


this_path = Path().resolve()
data_path = this_path.parent.parent / 'acouslic-ai-train-set'
assert data_path.exists()

In [2]:
sample_image = '0d0a3298-a9c6-43c3-a9e3-df3a9c0afa06.mha'
img_path = data_path / 'images/stacked_fetal_ultrasound/'
mask_path = data_path / 'masks/stacked_fetal_abdomen/'
im = sitk.ReadImage(str(img_path / sample_image))
mask = sitk.ReadImage(str(mask_path / sample_image))
im_array = sitk.GetArrayFromImage(im)
print(im_array.shape)
mask_array = sitk.GetArrayFromImage(mask)

(840, 562, 744)


In [3]:
is_stack = np.any(im_array, axis=(1, 2))
print(len(is_stack), sum(is_stack))
contains_mask = np.any(mask_array, axis=(1, 2))
print(len(contains_mask), sum(contains_mask))

840 670
840 35


In [9]:
np.where(is_stack)[0]

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 140, 141,
       142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154,
       155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167,
       168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180,
       181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193,
       194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 20

In [15]:
im_array[128].any(), im_array[59].any(), im_array[0].any()

(False, True, True)

In [29]:
np.any(im_array.reshape(840, -1), axis=0).shape

(418128,)

In [ ]:
w, h = im_array.size
x = 0
y = 0
width = 10
height = 10
boxes = [x, y, width, height]#[obj["bbox"] for obj in anno]
# guard against no boxes via resizing
boxes = torch.as_tensor(boxes, dtype=torch.float32).reshape(-1, 4)
boxes[:, 2:] += boxes[:, :2]
boxes[:, 0::2].clamp_(min=0, max=w)
boxes[:, 1::2].clamp_(min=0, max=h)